In this example, we will build Exa Researcher, a JavaScript app that, given a research topic, automatically searches for relevant sources with Exa's **Auto Search** and synthesizes the information into a reliable research report.

You can play with the code in your browser with this Replit [template](https://replit.com/@olafblitz/exa-researcher?v=1).

Alternatively, this [interactive notebook](https://github.com/exa-labs/exa-js/tree/master/examples/researcher/researcher.ipynb) was made with the Deno Javascript kernel for Jupyter so you can easily run it locally. Check out the [plain JS version](https://github.com/exa-labs/exa-js/tree/master/examples/researcher/researcher.mjs) if you prefer a regular Javascript file you can run with NodeJS, or want to skip to the final result. If you'd like to run this notebook locally, [Installing Deno](https://docs.deno.com/runtime/manual/getting_started/installation) and [connecting Deno to Jupyter](https://docs.deno.com/runtime/manual/tools/jupyter) is fast and easy.

To play with this code, first we need a [Exa API key](https://dashboard.exa.ai/overview) and an [OpenAI API key](https://platform.openai.com/api-keys). Get 1000 free Exa searches per month just for [signing up](https://dashboard.exa.ai/overview)!

# Setup

Let's import the Exa and OpenAI SDKs and put in our API keys to create a client object for each. Make sure to pick the right imports for your runtime and paste or load your API keys.

In [12]:
// Deno imports
import Exa from 'npm:exa-js';
import OpenAI from 'npm:openai';

// NodeJS imports
//import Exa from 'exa-js';
//import OpenAI from 'openai';


const EXA_API_KEY = "API key here" // insert or load your API key here
const OPENAI_API_KEY = "API key here"// insert or load your API key here

const exa = new Exa(EXA_API_KEY);
const openai = new OpenAI({ apiKey: OPENAI_API_KEY });

Since we'll be making several calls to the OpenAI API to get a completion from GPT-3.5 Turbo, let's make a simple utility function so we can pass in the system and user messages directly, and get the LLM's response back as a string.

In [3]:
async function getLLMResponse({system = 'You are a helpful assistant.', user = '', temperature = 1, model = 'gpt-3.5-turbo'}){
    const completion = await openai.chat.completions.create({
        model,
        temperature,
        messages: [
            {'role': 'system', 'content': system},
            {'role': 'user', 'content': user},
        ]
    });
    return completion.choices[0].message.content;
}

Okay, great! Now let's starting building Exa Researcher.

# Exa Auto Search

The app should be able to automatically generate research reports for all kinds of different topics. Here's two to start:

In [4]:
const SAMA_TOPIC = 'Sam Altman';
const ART_TOPIC = 'renaissance art';

The first thing our researcher has to do is decide what kind of search to do for the given topic. 

Exa offers two kinds of search: **neural** and **keyword** search. Here's how we decide:

- Neural search is preferred when the query is broad and complex because it lets us retrieve high quality, semantically relevant data. Neural search is especially suitable when a topic is well-known and popularly discussed on the Internet, allowing the machine learning model to retrieve contents which are more likely recommended by real humans.  
- Keyword search is useful when the topic is specific, local or obscure. If the query is a specific person's name, and identifier, or acronym, such that relevant results will contain the query itself, keyword search may do well. And if the machine learning model doesn't know about the topic, but relevant documents can be found by directly matching the search query, keyword search may be necessary.

Conveniently, Exa's Auto Search feature (on by default) will automatically decide whether to use `keyword` or `neural` search for each query. For example, if a query is a specific person's name, Exa would decide to use keyword search.

Now, we'll create a helper function to generate search queries for our topic.

In [5]:
async function generateSearchQueries(topic, n){
    const userPrompt = `I'm writing a research report on ${topic} and need help coming up with diverse search queries.
                        Please generate a list of ${n} search queries that would be useful for writing a research report on ${topic}. These queries can be in various formats, from simple keywords to more complex phrases. Do not add any formatting or numbering to the queries.`;

    const completion = await getLLMResponse({
        system: 'The user will ask you to help generate some search queries. Respond with only the suggested queries in plain text with no extra formatting, each on its own line.',
        user: userPrompt,
        temperature: 1
    });
    return completion.split('\n').filter(s => s.trim().length > 0).slice(0, n);
}


Next, let's write another function that actually calls the Exa API to perform searches using Auto Search

In [6]:
async function getSearchResults(queries, linksPerQuery=2){
    let results = [];
    for (const query of queries){
        const searchResponse = await exa.searchAndContents(query, { 
            numResults: linksPerQuery, 
            useAutoprompt: false 
        });
        results.push(...searchResponse.results);
    }
    return results;
}

# Writing a report with GPT-4

The final step is to instruct the LLM to synthesize the content into a research report, including citations of the original links. We can do that by pairing the content and the URLs and writing them into the prompt.

In [7]:
async function synthesizeReport(topic, searchContents, contentSlice = 750){
    const inputData = searchContents.map(item => `--START ITEM--\nURL: ${item.url}\nCONTENT: ${item.text.slice(0, contentSlice)}\n--END ITEM--\n`).join('');
    return await getLLMResponse({
        system: 'You are a helpful research assistant. Write a report according to the user\'s instructions.',
        user: 'Input Data:\n' + inputData + `Write a two paragraph research report about ${topic} based on the provided information. Include as many sources as possible. Provide citations in the text using footnote notation ([#]). First provide the report, followed by a single "References" section that lists all the URLs used, in the format [#] <url>.`,
        //model: 'gpt-4' //want a better report? use gpt-4 (but it costs more)
    });
}

# All Together Now

Now, let's just wrap everything into one Researcher function that strings together all the functions we've written. Given a user's research topic, the Researcher will generate search queries, feed those queries to Exa Auto Search, and finally use an LLM to synthesize the retrieved information. Three simple steps!

In [8]:
async function researcher(topic){
    console.log(`Starting research on topic: "${topic}"`);
    
    const searchQueries = await generateSearchQueries(topic, 3);
    console.log("Generated search queries:", searchQueries);
    
    const searchResults = await getSearchResults(searchQueries);
    console.log(`Found ${searchResults.length} search results. Here's the first one:`, searchResults[0]);
    
    console.log("Synthesizing report...");
    const report = await synthesizeReport(topic, searchResults);
    
    return report;
}

In just a couple lines of code, we've used Exa to go from a research topic to a valuable essay with up-to-date sources

In [10]:
console.log("Researching Sam Altman:");
const samaReport = await researcher(SAMA_TOPIC);
console.log("Sam Altman Report:");
console.log(samaReport);

Starting research on topic: "Sam Altman"
Generated search queries: [
  "Sam Altman biography",
  "Sam Altman Y Combinator contributions",
  "Sam Altman latest projects and investments"
]
Synthesizing report...
**Research Report on Sam Altman**

Sam Altman, born Samuel H. Altman on April 22, 1985, is a prominent American entrepreneur, investor, programmer, and blogger[^1]. Altman is widely known for his roles as the CEO of OpenAI and the former president of Y Combinator. He is recognized for his contributions to the tech industry and his expertise in business and technology ventures. Altman's passion for entrepreneurship and innovation is evident in his work, as he has been involved in various successful projects such as Loopt and OpenAI[^2]. His diverse experiences and leadership roles in prominent organizations showcase his strategic vision and commitment to pushing boundaries in the tech world.

Additionally, Altman has been actively engaged in the startup ecosystem, providing valuab

In [11]:
console.log("\n\nResearching Renaissance Art:");
const artReport = await researcher(ART_TOPIC);
console.log("Renaissance Art Report:");
console.log(artReport);

Starting research on topic: "renaissance art"
Generated search queries: [
  "renaissance art characteristics",
  "influential renaissance artists",
  "renaissance art techniques and materials"
]
Synthesizing report...
**Research Report on Renaissance Art:**

The Renaissance period, spanning from the 14th to the 17th centuries, was a transformative era in the realms of art, painting, sculpture, architecture, music, and literature across Europe. Characterized by a renewed emphasis on nature, a revival of classical learning, and a more individualistic approach towards man[#1], Renaissance art marked a departure from medieval artistic conventions. Artists during this time found inspiration in the classical art of ancient Greece and Rome, while also embracing scientific exploration to develop new techniques such as perspective image rendering for three-dimensional effects on flat surfaces[#2]. The shift towards humanism played a crucial role in liberating artists from the restrictions impos